In [ ]:
import sys, os
from collections import deque

import h5py

from aliases import *

In [ ]:
osjoin = os.path.join
def get_framepaths(datadir):
    frmPaths = []
    subDirs = [
        p for p in (osjoin(datadir, n) for n in os.listdir(datadir))
            if os.path.isdir(p)
        ]
    for subDir in subDirs:
        frmPaths.extend(
            osjoin(subDir, n) for n in os.listdir(subDir) if (n.endswith('.frm'))
            )
    return frmPaths

In [ ]:
os.path.basename('/home/morpheus/workspace/mount/data/charon5/linear2.frm')

In [ ]:
osrel = os.path.relpath

In [ ]:
osrel('/home/morpheus/workspace/mount/data/charon5/linear2.frm', datadir)

In [ ]:
get_framepaths(datadir).index('/home/morpheus/workspace/mount/data/pluto1/.frm')

In [ ]:
with h5py.File(os.path.join(datadir, 'pluto1', 'linear2.frm'), mode = 'r') as h5file:
    print(h5file.keys())

In [ ]:
frmPaths = []
subDirs = [
    p for p in (os.path.join(datadir, n) for n in os.listdir(datadir))
        if os.path.isdir(p)
    ]
for subDir in subDirs:
    frmPaths.extend(
        os.path.join(subDir, n) for n in os.listdir(subDir) if (n.endswith('.frm'))
        )

In [ ]:
h5py.Group.copy

In [ ]:
h5py.ExternalLink

In [ ]:
'_string_foo'[8:]

In [ ]:
%%time

observees = dict()

frmPath = frmPaths[0]
with h5py.File('temp.frm', mode = 'w') as writefile:
    with h5py.File(frmPath, mode = 'r') as readfile:
        for i, (key, grp) in enumerate(readfile.items()):
            if not i % 1000:
                print('.')
            try:
                styp = grp.attrs['supertype'][8:]
                if styp == 'System':
                    wgrp = writefile.require_group(key)
                    wgrp.attrs.update(grp.attrs)
                    wgrp['inputs'] = h5py.ExternalLink(frmPath, os.path.join(key, 'inputs'))
                elif styp == 'Observer':
                    typ = grp.attrs['type'][8:]
                    wgrp = writefile.require_group(grp['inputs'].attrs['observee'][7:])
                    okey = typ.lower()
                    if okey in wgrp:
                        if len(wgrp[okey]['count']) > len(grp['outputs']['count']):
                            continue
                    wgrp[okey] = h5py.ExternalLink(frmPath, os.path.join(key, 'outputs'))
            except (KeyError, RuntimeError):
                continue

In [ ]:
myfile = h5py.File('temp.frm', mode = 'r')

In [ ]:
myfile['aabrspuugh-eistiopoov']

In [ ]:
myfile.close()

In [ ]:
%%time
with h5py.File('temp.frm', mode = 'r') as h5file:
    print(len(h5file))
    out = h5file['aabrspuugh-eistiopoov']['thermo']['Nu'][()]

In [ ]:
# if not __name__ == '__main__':
#     raise RuntimeError

# import os
# from collections import deque

# import h5py

# from aliases import *

# frmPaths = []
# subDirs = [
#     p for p in (os.path.join(datadir, n) for n in os.listdir(datadir))
#         if os.path.isdir(p)
#     ]
# for subDir in subDirs:
#     frmPaths.extend(
#         os.path.join(subDir, n) for n in os.listdir(subDir) if (n.endswith('.frm'))
#         )

# with h5py.File(os.path.join(datadir, 'allout.frm'), mode = 'w') as writefile:
#     for frmPath in frmPaths:
#         print('-' * 10 + frmPath + '-' * 10)
#         with h5py.File(frmPath, mode = 'r') as readfile:
#             for i, (key, grp) in enumerate(readfile.items()):
#                 grpkeys = grp.keys()
#                 try:
#                     if not grp.attrs['supertype'] == '_string_Observer':
#                         continue
#                 except KeyError:
#                     continue
#                 if not i % 1000:
#                     print(key)
#                 try:
#                     readout = grp['outputs']
#                 except KeyError:
#                     continue
#                 hashID = grp.attrs['hashID'][8:]
#                 writegrp = writefile.require_group(hashID)
#                 systemgrp = readfile[hashID]
#                 writegrp.attrs.update(systemgrp.attrs)
#                 try:
#                     writefile.copy(systemgrp['inputs'], writegrp)
#                 except RuntimeError:
#                     pass
#                 outgrp = writegrp.require_group('outputs')
#                 for dname, dset in readout.items():
#                     if dname in outgrp:
#                         continue
#                     if not dset:
#                         continue
#                     try:
#                         oset = outgrp.create_dataset(
#                             dname,
#                             data = dset[()],
#                             chunks = dset.shape
#                             )
#                     except OSError:
#                         try:
#                             oset = outgrp.create_dataset(
#                                 dname,
#                                 data = (sub := dset[:-10]),
#                                 chunks = sub.shape
#                                 )
#                         except OSError:
#                             continue

In [ ]:
#         for i, (key, grp) in enumerate(readfile.items()):
#             if not i % 1000:
#                 print('.')
#             try:
#                 observees[key] = grp['inputs'].attrs['observee'][7:]
#             except KeyError:
#                 continue

In [ ]:
def visit(key, grp):
    try:
        if not grp.attrs['supertype'] == '_string_Observer':
            continue
    except KeyError:
        return
    try:
        systemid = grp['inputs']['observee'][7:]
    except KeyError:
        return
    systemid

In [ ]:
    for i, (key, grp) in enumerate(readfile.items()):
        grpkeys = grp.keys()
        try:
            if not grp.attrs['supertype'] == '_string_Observer':
                continue
        except KeyError:
            continue
        if not i % 1000:
            print(key)
        try:
            readout = grp['outputs']
        except KeyError:
            continue
        hashID = grp.attrs['hashID'][8:]
        writegrp = writefile.require_group(hashID)
        systemgrp = readfile[hashID]
        writegrp.attrs.update(systemgrp.attrs)
        try:
            writefile.copy(systemgrp['inputs'], writegrp)
        except RuntimeError:
            pass
        outgrp = writegrp.require_group('outputs')
        for dname, dset in readout.items():
            if dname in outgrp:
                continue
            if not dset:
                continue
            try:
                oset = outgrp.create_dataset(
                    dname,
                    data = dset[()],
                    chunks = dset.shape
                    )
            except OSError:
                try:
                    oset = outgrp.create_dataset(
                        dname,
                        data = (sub := dset[:-10]),
                        chunks = sub.shape
                        )
                except OSError:
                    continue

In [ ]:
frmPaths[0]

In [ ]:
with h5py.File(frmPaths[0], mode = 'r') as h5file:
    print(list(h5file['xoosftzii-skorhgleuv'].items()))

In [ ]:
testpath = frmPaths[0]

In [ ]:
%%timeit
bool(1 % 1000)

In [ ]:
help(h5py.Dataset.write_direct)

In [ ]:
help(h5py.Group.create_dataset)

In [ ]:
help(h5py.Dataset.read_direct)

In [ ]:
help(h5py.Group.copy)

In [ ]:
ah5py.File.copy()

In [ ]:
h5py.copy

In [ ]:
with h5py.File('temp.frm', mode = 'w') as writefile:
    with h5py.File(os.path.join(testpath), mode = 'r') as readfile:
        for i, (key, grp) in enumerate(readfile.items()):
            if i > 1000:
                print('.')
            grpkeys = grp.keys()
            if not 'observee' in grpkeys:
                continue
            hashID = grp.attrs['hashID'][8:]
            writegrp = writefile.require_group(hashID)
            writefile.copy(readfile[hashID]['inputs'], writegrp)
            readfile.copy(grp['outputs'], writegrp, name = grp.attrs['type'])

In [ ]:
len(keys)

In [ ]:
with h5py.Fil(os.path.join(datadir, 'out.frm'), mode = 'r') as writefile:
    with h5py.File(os.path.join(datadir, 'mergedcopy.frm'), mode = 'r') as readfile:
        

In [ ]:
reader = Reader('mergedcopy', datadir)

In [ ]:
i = 0
out = []
def visitprint(name):
    global i
    global out
    i += 1
    if i > 100:
        return True
    out.append(name)

In [ ]:
out = []
with h5py.File(os.path.join(datadir, 'mergedcopy.frm'), mode = 'r') as readfile:
#     readfile.visit(visitprint)
    readfile.visit(out.append)

In [ ]:
len(out)

In [ ]:
links = []
with h5py.File(os.path.join(datadir, 'mergedcopy.frm'), mode = 'r') as readfile:
    readfile.id.links.iterate(links.append)

In [ ]:
datasets = dict()
with h5py.File(os.path.join(datadir, 'merged.frm'), mode = 'r') as readfile:
    links = deque()
    readfile.id.links.iterate(links.append)
    subpaths = deque()
    for i, link in enumerate(links):
        if not i % 1000:
            print('.')
        subpaths.clear()
        readfile[link].visit(subpaths.append)
        if 'observee' in subpaths:
            target = datasets.setdefault(
                readfile[link]['observee'].attrs['hashID'][8:],
                deque()
                )
            linkmaker = lambda x: target.append(os.path.join(link.decode(), 'outputs', x))
            readfile[link]['outputs'].visit(linkmaker)